In [ ]:
%matplotlib widget
from mmdet3d.datasets import build_dataset
from tools.misc.browse_dataset import build_data_cfg
from mmdet3d.models import apply_3d_transformation
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
import copy
import torch
import numpy as np
from mmdet3d.ops.voxel.voxelize import voxelization

In [ ]:
def proj_points_to_2d(points, proj_mat):
    points = np.concatenate([points[:, :3].reshape(-1, 3), np.ones((points.shape[0], 1))], axis=-1)    
    points_2d = points @ proj_mat.T
    points_2d[:, 2] = np.clip(points_2d[:, 2], a_min=1e-5, a_max=1e5)
    points_2d[:, 0] /= points_2d[:, 2]
    points_2d[:, 1] /= points_2d[:, 2]
    return points_2d

def proj_point_to_2d(point, proj_mat):
    point = np.concatenate([point, [1]], axis=-1)    
    point_2d = point @ proj_mat.T
    point_2d[2] = np.clip(point_2d[2], a_min=1e-5, a_max=1e5)
    point_2d[0] /= point_2d[2]
    point_2d[1] /= point_2d[2]
    return point_2d

def get_rgb(img):
    # Convert img to RGB
    img = img.int()
    img = img.flip(0) # BGR image, flip to RGB
    return img.permute(1, 2, 0) # Permute to (H, W, C)

### Build dataset from config

In [ ]:
config = "configs/_base_/datasets/nus-3d-1sweep-remove_close_multi_modal_input_single_sample.py"
cfg = build_data_cfg(config, ["Normalize"], None)
try:
    dataset = build_dataset(cfg.data.train, default_args=dict(filter_empty_gt=False))
except TypeError:  # seg dataset doesn't have `filter_empty_gt` key
    dataset = build_dataset(cfg.data.train)

### Read data sample

In [ ]:
cam_idx = 0
sample = dataset[0]
img_metas = sample["img_metas"].data
points = sample["points"].data
imgs = sample["img"].data
img = imgs[cam_idx]
rgb = get_rgb(img)
proj_mat = img_metas['lidar2img'][cam_idx]

### Project lidar points to img

In [ ]:
proj_mat = copy.deepcopy(proj_mat).reshape(4, 4)
if isinstance(proj_mat, torch.Tensor):
    proj_mat = proj_mat.cpu().numpy()

points_2d = proj_points_to_2d(points, proj_mat)

is_point_in_image = [(0 <= p[0] <= 1600) and (0 <= p[1] <= 900) for p in points_2d] 
points_2d_in_image = points_2d[is_point_in_image, :]
points_3d_in_image = points[is_point_in_image, :]

### Plot img and lidar points

In [ ]:
# Create subplot with rgb image, 3d points and projected lidar points
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(rgb)
ax[1].plot(points_3d_in_image[:, 0], points_3d_in_image[:, 1], ".")
ax[1].axis("equal")
ax[2].imshow(rgb)
ax[2].scatter(points_2d_in_image[:, 0], points_2d_in_image[:, 1], s=1, c=points_2d_in_image[:, 2], cmap="jet")
plt.show()

# Create 3d plot of lidar points
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points_3d_in_image[:, 0], points_3d_in_image[:, 1], points_3d_in_image[:, 2], s=1, c=points_3d_in_image[:, 2], cmap="jet")
ax.set_box_aspect((np.ptp(points_3d_in_image[:, 0]), np.ptp(points_3d_in_image[:, 1]), np.ptp(points_3d_in_image[:, 2])))
plt.show()


### Voxelize point cloud

In [ ]:
voxel_size = 1.0
r = 50
point_cloud_range = (-r, -r, -5, r, r, 3)
voxels, voxel_coors, num_points_per_voxel = voxelization(points, (voxel_size, voxel_size, 8), point_cloud_range)
n_voxels_x = (point_cloud_range[3] - point_cloud_range[0])/voxel_size

### Plot voxel grid and non-empty voxels

In [ ]:
plt.figure(figsize=(7, 7))
plt.plot(voxel_coors[:, 2]-n_voxels_x/2, voxel_coors[:, 1]-n_voxels_x/2, ".")
plt.xticks(np.arange(-n_voxels_x/2, n_voxels_x/2 + 1, 1) - 0.5)
plt.yticks(np.arange(-n_voxels_x/2, n_voxels_x/2 + 1, 1) - 0.5)
plt.grid(True, which='major')
plt.show()

### Lidar points visible in different camera views

In [ ]:

plt.figure(figsize=(7, 7))

sample = dataset[0]
points = sample["points"].data
img_metas = sample["img_metas"].data
for cam_idx in range(6):
    proj_mat = img_metas['lidar2img'][cam_idx]
    
    proj_mat = copy.deepcopy(proj_mat).reshape(4, 4)
    if isinstance(proj_mat, torch.Tensor):
        proj_mat = proj_mat.cpu().numpy()
        
    points_2d = proj_points_to_2d(points, proj_mat)

    is_point_in_image = [(0 <= p[0] <= 1600) and (0 <= p[1] <= 900) for p in points_2d] 
    points_3d_in_image = points[is_point_in_image, :]
    
    plt.plot(points_3d_in_image[:, 0], points_3d_in_image[:, 1], ".", label=f"Cam: {cam_idx}")

    
plt.axis("equal")
plt.legend()
plt.show()

# Reset cam idx to front
cam_idx = 0
img = imgs[cam_idx]
proj_mat = img_metas['lidar2img'][cam_idx]

### Front cam voxels

In [ ]:
from matplotlib import patches

# Projection points to 2d
proj_mat = copy.deepcopy(proj_mat).reshape(4, 4)
if isinstance(proj_mat, torch.Tensor):
    proj_mat = proj_mat.cpu().numpy()

points_2d = proj_points_to_2d(points, proj_mat)

is_point_in_image = [(0 <= p[0] <= 1600) and (0 <= p[1] <= 900) for p in points_2d] 
points_2d_in_image = points_2d[is_point_in_image, :]
points_3d_in_image = points[is_point_in_image, :]

# Voxelize
voxel_size = 0.5
r = 50
#point_cloud_range = (-r, 0, -5, r, r, 3)
point_cloud_range = (-r, -r, -5, r, r, 3)
voxels, voxel_coors, num_points_per_voxel = voxelization(points_3d_in_image, (voxel_size, voxel_size, 8), point_cloud_range)
#voxels, voxel_coors, num_points_per_voxel = voxelization(points, (voxel_size, voxel_size, 8), point_cloud_range)
n_voxels_x = (point_cloud_range[3] - point_cloud_range[0])/voxel_size

# Create two subplots of voxel grid and voxel points
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.set_title("Voxel grid")
ax1.plot(voxel_coors[:, 2], voxel_coors[:, 1], ".")
ax1.set_xticks(np.arange(0, n_voxels_x + 1, 1) - 0.5)
ax1.set_yticks(np.arange(0, n_voxels_x + 1, 1) - 0.5)
ax1.grid(True, which='major')
ax1.axis("equal")

ax2.set_title("Voxel points")
ax2.plot(voxels[:, :, 0], voxels[:, :, 1], ".", c="b")
ax2.axis("equal")
# Plot square for the point cloud range
ax2.add_patch(
    patches.Rectangle(
        (point_cloud_range[0], point_cloud_range[1]),
        point_cloud_range[3]-point_cloud_range[0],
        point_cloud_range[4]-point_cloud_range[1],
        linewidth=1, edgecolor='r', facecolor='none'
    )
)
plt.show()

print("Voxel size: ", voxel_size)
print("Point cloud range: ", point_cloud_range)
print("Voxel grid shape: ", voxels.shape)

# Create subplots with histograms for x, y, z in voxels
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
ax1.hist(voxels[:, :, 0].flatten(), bins=100)
ax1.set_title("X")
ax1.set_ylim(0, 1000)
ax2.hist(voxels[:, :, 1].flatten(), bins=100)
ax2.set_title("Y")
ax2.set_ylim(0, 1000)
ax3.hist(voxels[:, :, 2].flatten(), bins=100)
ax3.set_title("Z")
ax3.set_ylim(0, 1000)
plt.show()


### Voxel projection methods
- Center of mass (avg(x), avg(y), avg(z) or median)
- Voting strategy (choose the window where most of the points end up)
- Projection of all points (voxel feature end up in all windows that points project to)
- Center coordinate
- Corners of the voxel

In [ ]:
def center_of_mass(voxel):
    non_zero_points = sum([ point.any() for point in voxel ])

    if non_zero_points == 0:
        return np.array([0, 0, 0])

    voxel_mean_x = voxel[:, 0].sum() / non_zero_points
    voxel_mean_y = voxel[:, 1].sum() / non_zero_points
    voxel_mean_z = voxel[:, 2].sum() / non_zero_points

    return np.array([voxel_mean_x, voxel_mean_y, voxel_mean_z])

#### Creating voxel grid data structure for easier plotting of selected voxels

In [ ]:
# Create voxel grid
n_voxels_y = n_voxels_x
n_voxels_z = 1
voxel_grid = np.zeros((int(n_voxels_x), int(n_voxels_y), int(n_voxels_z), voxels.shape[1], voxels.shape[2]))

# Fill voxel grid with voxels
for i in range(voxel_coors.shape[0]):
    voxel_grid[voxel_coors[i, 2], voxel_coors[i, 1], voxel_coors[i, 0], :, :] = voxels[i, :, :]


#### Clickable plot for visualizing projection of different voxels

In [ ]:
# Image plane windows
window_shape_image = (200, 200, 1)
window_shape_lidar = (16, 16, 1)

# Plot grid lines
fig, axs = plt.subplots(1, 2, figsize=(15, 10))
axs[0].xaxis.set_major_locator(MultipleLocator(voxel_size * window_shape_lidar[0]))
axs[0].xaxis.set_major_formatter(FormatStrFormatter('%d'))
axs[0].xaxis.set_minor_locator(MultipleLocator(voxel_size))
axs[0].yaxis.set_major_locator(MultipleLocator(voxel_size * window_shape_lidar[1]))
axs[0].yaxis.set_minor_locator(MultipleLocator(voxel_size))
axs[0].xaxis.grid(True,'minor')
axs[0].yaxis.grid(True,'minor')
axs[0].xaxis.grid(True,'major',linewidth=2)
axs[0].yaxis.grid(True,'major',linewidth=2)
axs[0].axis("equal")

#axs[1].set_xticks( np.arange(0, 1600, window_shape[0]))
#axs[1].set_yticks( np.arange(0, 900, window_shape[1]))
axs[1].xaxis.set_major_locator(MultipleLocator(window_shape_image[0]))
axs[1].xaxis.set_major_formatter(FormatStrFormatter('%d'))
axs[1].yaxis.set_major_locator(MultipleLocator(window_shape_image[1]))
axs[1].xaxis.grid(True,'major',linewidth=2)
axs[1].yaxis.grid(True,'major',linewidth=2)

# Plot voxel points stored in voxel grid
for i in range(voxel_grid.shape[0]):
    for j in range(voxel_grid.shape[1]):
        for k in range(voxel_grid.shape[2]):
            voxel = voxel_grid[i, j, k, :, :]
            axs[0].plot(voxel[:, 0], voxel[:, 1], ".", c="b")

axs[1].imshow(rgb)

# Create clickable plot
def onclick(event):
    x = event.xdata
    y = event.ydata

    # Closest voxel coordinate
    x = int(np.floor(x))
    y = int(np.floor(y))
    
    # Get voxel
    vx = x + int(n_voxels_x/2)
    vy = y #+ int(n_voxels_y/2)
    voxel = voxel_grid[vx, vy, 0, :, :]

    # Get non-zero points in voxel
    voxel = voxel[voxel.any(axis=1)]

    # Plot mark in center of voxel
    x = x + voxel_size/2
    y = y + voxel_size/2
    axs[0].plot(x, y, "x", c="r")
    
    # Plot voxel points
    axs[0].plot(voxel[:, 0], voxel[:, 1], ".", c="r")

    # Compute center of mass
    if voxel.any():
        # Project voxel points to 2D
        voxel_2d = proj_points_to_2d(voxel, proj_mat)
        axs[1].plot(voxel_2d[:, 0], voxel_2d[:, 1], ".", c="b")

        # Compute center of mass
        voxel_com = center_of_mass(voxel)
        axs[0].plot(voxel_com[0], voxel_com[1], "o", c="g")

        # Project center of mass to 2D
        voxel_com_2d = proj_point_to_2d(voxel_com, proj_mat)
        axs[1].plot(voxel_com_2d[0], voxel_com_2d[1], "o", c="g")


    fig.canvas.draw()

cid = fig.canvas.mpl_connect('button_press_event', onclick)

plt.show()



#### Voxel centroids (center of mass)

In [ ]:
# Get number of non-zero points for each voxel
non_zero_points_per_voxel = [ sum([ point.any() for point in voxel ]) for voxel in voxels ]
plt.figure(figsize=(5, 5))
plt.hist(non_zero_points_per_voxel, bins=100)
plt.title("Number of non-zero points per voxel")
plt.show()

# Compute center of mass for each voxel
voxel_centroids = np.zeros((voxels.shape[0], 3))
for i, voxel in enumerate(voxels):
    voxel_mean_x = voxel[:, 0].sum() / non_zero_points_per_voxel[i]
    voxel_mean_y = voxel[:, 1].sum() / non_zero_points_per_voxel[i]
    voxel_mean_z = voxel[:, 2].sum() / non_zero_points_per_voxel[i]
    voxel_centroids[i, :] = [voxel_mean_x, voxel_mean_y, voxel_mean_z]


# Project voxel centroids to 2d
voxel_centroids_2d = proj_points_to_2d(voxel_centroids, proj_mat)


# Create subplot lidar points, projected lidar points, voxel centroids and projected voxel centroids
fig, ax = plt.subplots(1, 4, figsize=(17, 5))
ax[0].plot(points_3d_in_image[:, 0], points_3d_in_image[:, 1], ".")
ax[0].axis("equal")
ax[1].imshow(rgb)
ax[1].scatter(points_2d_in_image[:, 0], points_2d_in_image[:, 1], s=1, c=points_2d_in_image[:, 2], cmap="jet")
ax[2].plot(voxel_centroids[:, 0], voxel_centroids[:, 1], ".")
ax[2].axis("equal")
ax[3].imshow(rgb)
ax[3].scatter(voxel_centroids_2d[:, 0], voxel_centroids_2d[:, 1], s=1, c=voxel_centroids_2d[:, 2], cmap="jet")
plt.show()

# Create 3d plot of lidar points
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(121, projection='3d')
ax.scatter(points_3d_in_image[:, 0], points_3d_in_image[:, 1], points_3d_in_image[:, 2], s=1, c=points_3d_in_image[:, 2], cmap="jet")
ax.set_box_aspect((np.ptp(points_3d_in_image[:, 0]), np.ptp(points_3d_in_image[:, 1]), np.ptp(points_3d_in_image[:, 2])))

# Create 3d plot of voxel centroids
ax = fig.add_subplot(122, projection='3d')
ax.scatter(voxel_centroids[:, 0], voxel_centroids[:, 1], voxel_centroids[:, 2], s=1, c=voxel_centroids[:, 2], cmap="jet")
ax.set_box_aspect((np.ptp(voxel_centroids[:, 0]), np.ptp(voxel_centroids[:, 1]), np.ptp(voxel_centroids[:, 2])))
plt.show()
